In [33]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import warnings
warnings.filterwarnings('ignore')

# all import statements
import numpy as np
import pandas as pd
import pydicom as pyd
import os
import matplotlib.pyplot as plt
import mudicom

from os.path import dirname, join
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from PIL import Image

from scipy.misc import imresize
from scipy.signal import convolve2d
import scipy
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

from numpy import newaxis

from skimage.measure import shannon_entropy
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy


import math
from skimage.measure import label, regionprops

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import matplotlib.patches as mpatches

import statistics

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

import cv2
from skimage import img_as_float
from skimage import exposure

In [3]:
import pickle

from tqdm import tqdm
import os
import sys

In [4]:
def load_pickle(fileName):
    with open(fileName, "rb") as fp:
        file = pickle.load(fp)
    return file 

def write_pickle(file, fileName):
    with open(fileName, "wb") as fp:
        pickle.dump(file, fp)

### GLCM

In [5]:

def s_entropy(dcm):
    return shannon_entropy(dcm)

def entropy_simple(dcm):
    return entropy(dcm)

class glcm:
    def __init__(self, image):
        distance = [1, 2, 3]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        self.image = img_as_ubyte(image.astype('int64'))
        self.glcm_mat = greycomatrix(self.image, distances = distance, angles = angles, symmetric = True, normed = True)
        self.properties = ['correlation', 'homogeneity', 'contrast', 'energy']
            
    def correlation(self):
        return greycoprops(self.glcm_mat, 'correlation').flatten()
    
    def homogeneity(self):
        return greycoprops(self.glcm_mat, 'homogeneity').flatten()
    
    def contrast(self):
        return greycoprops(self.glcm_mat, 'contrast').flatten()
    
    def energy(self):
        return greycoprops(self.glcm_mat, 'energy').flatten()
    
    def glcm_all(self):
        return np.hstack([greycoprops(self.glcm_mat, props).ravel() for props in self.properties])
    

### Region Props

In [6]:
# thresh_ada = threshold_otsu(image_adahist)
# bw_ada = closing(image_adahist > thresh_ada, square(3))
# bw_clear_ada = clear_border(bw_ada)

class region_props:
    def __init__(self, image):
        self.image = image
        self.thresh = threshold_otsu(self.image)
        self.bw = closing(self.image > self.thresh, square(3))
        self.bw_clear = clear_border(self.bw)
        self.bw_label = label(self.bw_clear)
        self.regions = regionprops(self.bw_label)
#         self.idx = 0
        
    def plot_image_bw(self):
        plt.imshow(self.bw, cmap = plt.cm.bone)
    
    def plot_image(self):
        plt.imshow(self.bw_clear, cmap=plt.cm.bone)
        
    def plot_image_with_labels(self):
        plt.imshow(self.bw_label, cmap=plt.cm.bone)
        
    def max_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return max(lista)
    
    def eccentricity(self):
        
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].eccentricity
    
    def euler_number(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].euler_number
    
    def solidity(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].solidity
    
    def perimeter(self):
        lista = []
        for e in self.regions:
            lista.append(e.perimeter)
        return max(lista)
    
    def mean_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        return statistics.mean(lista)
    
    def std_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>2:            
            return statistics.stdev(lista)
        else:
            return -1
    
    def thresh_img(self):
        return self.thresh
        

### Moments

In [7]:
class moments:
    
    def __init__(self, image):
        self.image = image
        self.moment = cv2.moments(self.image)
        self.hu = cv2.HuMoments(self.moment)
        
    def get_moments(self):
#         keys = [key for key in self.moment.keys()]
        values = [value for value in self.moment.values()]
        return values
    
    def get_HuMoments(self):
        moments_hu = []
        for m in range(len(self.hu)):
            moments_hu.append(self.hu[m][0])
        return moments_hu

#### DataFrame dcm_1

In [8]:
dcm_1 = load_pickle("dcm-1")
pat_1 = load_pickle("pat-1")

In [9]:
len(dcm_1), len(pat_1)

(4000, 4000)

In [10]:
data_dcm_1 = dict()
for dc in tqdm(range(len(dcm_1))):
    data_dcm_1[pat_1[dc]] = [s_entropy(dcm_1[dc]), entropy_simple(dcm_1[dc]), moments(dcm_1[dc]).get_moments(), moments(dcm_1[dc]).get_HuMoments(),
                      region_props(dcm_1[dc]).max_area(), region_props(dcm_1[dc]).eccentricity(), region_props(dcm_1[dc]).euler_number(),
                      region_props(dcm_1[dc]).solidity(), region_props(dcm_1[dc]).perimeter(), region_props(dcm_1[dc]).mean_area(), region_props(dcm_1[dc]).std_area(),
                      region_props(dcm_1[dc]).thresh_img(), glcm(dcm_1[dc]).correlation(), glcm(dcm_1[dc]).homogeneity(),
                      glcm(dcm_1[dc]).contrast(), glcm(dcm_1[dc]).energy()
                     ]

100%|██████████| 4000/4000 [56:31<00:00,  1.15it/s]


In [11]:
df_dcm_1 = pd.DataFrame(data_dcm_1).T

In [12]:
df_dcm_1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
7a1ddad0-eda0-4ae1-872f-50960e5abeff,6.32622,4.385,"([m00, m10, m01, m20, m11, m02, m30, m21, m12,...","[0.001221533083578024, 3.595201185958721e-08, ...",471436,0.55973,-49,0.775327,8745.16,7082,57589.5,80,"[0.9972485126459916, 0.9929092448137423, 0.994...","[0.7151632163120861, 0.6566048498165192, 0.711...","[30.5509301686217, 78.73314547422946, 57.35879...","[0.24865751457564536, 0.24619105493885057, 0.2..."
755c7169-e84a-4327-a4d6-b0ff906a88f8,7.71827,5.3499,"([m00, m10, m01, m20, m11, m02, m30, m21, m12,...","[0.001064567904782183, 5.920794592180254e-09, ...",1328,0.701463,0,0.397248,492.168,43,158.073,128,"[0.9956474877343438, 0.9907231531278154, 0.994...","[0.5113259006619656, 0.44208361797549245, 0.54...","[32.65148746792523, 69.39420694505364, 38.1463...","[0.03400775744752322, 0.030995100398568544, 0...."
f29e3380-0224-468d-b0b0-b2ba8892e7a5,7.55772,5.23861,"([m00, m10, m01, m20, m11, m02, m30, m21, m12,...","[0.0013010868405398067, 5.81922740371494e-09, ...",1256,0.994265,1,0.189356,549.428,31,118.975,122,"[0.9943552355212539, 0.988322981368471, 0.9928...","[0.5326860700454552, 0.4700566606383245, 0.576...","[33.59518668285678, 69.3264964468256, 42.41161...","[0.03492570452446144, 0.031674482210111106, 0...."
3537a092-af77-448a-9cfb-f28903403012,7.57827,5.25286,"([m00, m10, m01, m20, m11, m02, m30, m21, m12,...","[0.0010803192114857943, 7.876746356690139e-09,...",3826,0.921584,-2,0.570364,644.061,44,373.115,123,"[0.9987968130005396, 0.9978840416623213, 0.998...","[0.42730732507711416, 0.3565538436109494, 0.42...","[12.296345193365106, 21.624122217348983, 11.85...","[0.06956586400859374, 0.06793835826076827, 0.0..."
d9b76cc0-f23b-4a92-8cba-08627b829b87,7.55487,5.23663,"([m00, m10, m01, m20, m11, m02, m30, m21, m12,...","[0.0013881351613918894, 3.654127317591776e-08,...",852,0.558342,-1,0.403218,508.54,17,67.8159,104,"[0.9949999521331908, 0.9897872508093514, 0.993...","[0.4487954244464176, 0.3889849640301411, 0.494...","[29.647267152370464, 60.43948328235528, 35.634...","[0.03621358676684327, 0.03359038146995782, 0.0..."


In [14]:
df_dcm_1[2][1][1]

[139042842.0,
 61318717043.0,
 81624989695.0,
 36817454596803.0,
 35460491739282.0,
 58723545625355.0,
 2.53958116560567e+16,
 2.100978773108324e+16,
 2.527203988224677e+16,
 4.6237708674798e+16,
 9775537114999.094,
 -536612280970.2344,
 10805660411897.867,
 536956503622384.0,
 -130571287347571.0,
 4643994625078.0,
 -922810714334816.0,
 0.0005056422519163334,
 -2.7756412661911796e-05,
 0.0005589256528658496,
 2.3554154356248208e-06,
 -5.72764504374376e-07,
 2.0371364438412644e-08,
 -4.048004979995147e-06]

In [26]:
df_dcm_1.to_pickle('df_dcm_1')

In [27]:
df_dcm_1.to_csv('df_dcm_1.csv')

In [28]:
df_moments_series = df_dcm_1[2].apply(pd.Series)

In [29]:
df_moments_series.head()

,0,1
7a1ddad0-eda0-4ae1-872f-50960e5abeff,"[m00, m10, m01, m20, m11, m02, m30, m21, m12, ...","[77261405.0, 32664242311.0, 42314861159.0, 169..."
755c7169-e84a-4327-a4d6-b0ff906a88f8,"[m00, m10, m01, m20, m11, m02, m30, m21, m12, ...","[139042842.0, 61318717043.0, 81624989695.0, 36..."
f29e3380-0224-468d-b0b0-b2ba8892e7a5,"[m00, m10, m01, m20, m11, m02, m30, m21, m12, ...","[115788599.0, 57272718537.0, 69914628692.0, 36..."
3537a092-af77-448a-9cfb-f28903403012,"[m00, m10, m01, m20, m11, m02, m30, m21, m12, ...","[139943564.0, 72461632697.0, 85460124620.0, 48..."
d9b76cc0-f23b-4a92-8cba-08627b829b87,"[m00, m10, m01, m20, m11, m02, m30, m21, m12, ...","[114819170.0, 64448953239.0, 61807363488.0, 44..."


In [31]:
df_moments_series = df_moments_series[1].apply(pd.Series)

In [32]:
df_moments_series.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
7a1ddad0-eda0-4ae1-872f-50960e5abeff,77261405.0,3.266424e+10,4.231486e+10,1.699300e+13,1.756358e+13,2.728356e+13,1.000394e+16,9.015065e+15,1.117330e+16,1.908728e+16,...,-1.598626e+13,-4.307611e+12,-3.556842e+14,0.000533,-0.000055,0.000688,2.439902e-06,-3.046776e-07,-8.209751e-08,-6.778881e-06
755c7169-e84a-4327-a4d6-b0ff906a88f8,139042842.0,6.131872e+10,8.162499e+10,3.681745e+13,3.546049e+13,5.872355e+13,2.539581e+16,2.100979e+16,2.527204e+16,4.623771e+16,...,-1.305713e+14,4.643995e+12,-9.228107e+14,0.000506,-0.000028,0.000559,2.355415e-06,-5.727645e-07,2.037136e-08,-4.048005e-06
f29e3380-0224-468d-b0b0-b2ba8892e7a5,115788599.0,5.727272e+10,6.991463e+10,3.658900e+13,3.480176e+13,5.139891e+13,2.642027e+16,2.196953e+16,2.553614e+16,4.116142e+16,...,-3.407802e+14,-1.527898e+14,-9.642092e+14,0.000616,0.000016,0.000685,1.045125e-06,-2.362163e-06,-1.059083e-06,-6.683543e-06
3537a092-af77-448a-9cfb-f28903403012,139943564.0,7.246163e+10,8.546012e+10,4.894868e+13,4.443135e+13,6.191697e+13,3.710674e+16,2.967646e+16,3.227732e+16,4.876786e+16,...,-4.025385e+14,-3.597179e+12,-9.252947e+14,0.000584,0.000009,0.000497,-3.185318e-07,-1.737501e-06,-1.552672e-08,-3.993905e-06
d9b76cc0-f23b-4a92-8cba-08627b829b87,114819170.0,6.444895e+10,6.180736e+10,4.415926e+13,3.516904e+13,4.358788e+13,3.325940e+16,2.380690e+16,2.516871e+16,3.448443e+16,...,-4.985354e+14,1.899454e+14,-8.618152e+13,0.000606,0.000036,0.000783,-3.468627e-06,-3.529068e-06,1.344599e-06,-6.100678e-07
